In [1]:
import pandas as pd
import numpy as np
import cv2 
import mediapipe as mp
import time
import datetime
import matplotlib.pyplot as plt
import skvideo.io
import os
import tensorflow as tf

In [2]:
model=tf.keras.models.load_model(r"C:\Users\soory\Documents\GitHub\Project_Moorthi\checkpoint")

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 45, 7, 7, 1024)    3228864   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 45, 50176)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               25756160  
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1040      
Total params: 28,994,320
Trainable params: 28,972,432
Non-trainable params: 21,888
_________________________________________________________________


In [4]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

In [5]:
def pose_estimation(image,results):
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        return image

In [14]:
start=time.time()
i=0
while(i<5):
        cap = cv2.VideoCapture(0)
        out=cv2.VideoWriter("trier.mp4",cv2.VideoWriter_fourcc('M','J','P','G'), 10, (224,224))
        # Initiate holistic model
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                   break
                #print(i)
                #i=i+1
                # Recolor Feed

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Make Detections
                results = holistic.process(image)  
                # Recolor image back to BGR for rendering
                output = pose_estimation(image,results)
                output=cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
                out.write(output)
                #cv2.imshow('image2',output)
                if time.time() -start >3:
                    break
                if cv2.waitKey(10) & 0xFF == ord('q'):

                     break           
          #      cv2_imshow(image)

        cap.release()
        cv2.destroyAllWindows()
        
        
        i+=1   
        print(i)
        start=time.time()

1
2
3
4
5


In [61]:
cap = cv2.VideoCapture(0)
#out=cv2.VideoWriter("trier.mp4",cv2.VideoWriter_fourcc('M','J','P','G'), 10, (224,224))
        # Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                   break
                #print(i)
                #i=i+1
                # Recolor Feed

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # Make Detections
                results = holistic.process(image)  
                # Recolor image back to BGR for rendering
                output = pose_estimation(image,results)
                output=cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
                #out.write(output)
                cv2.imshow('image2',output)
              #  if time.time() -start >5:
               #     break
                if cv2.waitKey(10) & 0xFF == ord('q'):

                     break           
          #      cv2_imshow(image)

cap.release()
cv2.destroyAllWindows()

In [51]:
def video_array_maker(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
  outpath=os.path.join("./Output",pather)
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 20, (width,height))
  #start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    actualframe=len(videodata)
    print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                frame =cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
                results = holistic.process(frame)  

                output = pose_estimation(frame,results)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = pose_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
 #             arr.append(output)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              
#              arr.append(newframe)
              out.write(np.uint8(newframe))
    out.release()
    print("File Created : {}".format(outpath))
    return outpath

In [17]:
## Approach:

In [53]:
cap = cv2.VideoCapture(0)
width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out= cv2.VideoWriter('input.mp4', cv2.VideoWriter_fourcc(*'DIVX'),10, (width,height))
start=time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
       break 
    out.write(frame)
    if time.time() -start >2:
        break     
        
cap.release()        
out.release()      
cv2.destroyAllWindows()

outpath=video_array_maker("input.mp4")

model_input=skvideo.io.vread(outpath)
prediction=model.predict(np.expand_dims(model_input,axis=0))

Actual frame 51
File Created : ./Output\input.mp4


In [58]:
np.argmax(prediction,axis=1)

array([1], dtype=int64)

In [59]:
## For loop

## Checking out Model Performance with Real Time Data :

In [60]:
start=time.time()
i=0
while(i<5):
    cap = cv2.VideoCapture(0)
    width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out= cv2.VideoWriter('input.mp4', cv2.VideoWriter_fourcc(*'DIVX'),10, (width,height))
    start=time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
           break 
        out.write(frame)
        if time.time() -start >3:
            break     

    cap.release()        
    out.release()      
    cv2.destroyAllWindows()

    outpath=video_array_maker("input.mp4")

    model_input=skvideo.io.vread(outpath)
    prediction=model.predict(np.expand_dims(model_input,axis=0))
    arg_pred=np.argmax(prediction,axis=1)
    i+=1
    print("{} video is {}".format(i,arg_pred))
    start=time.time()

Actual frame 82
File Created : ./Output\input.mp4
1 video is [1]
Actual frame 81
File Created : ./Output\input.mp4
2 video is [1]
Actual frame 82
File Created : ./Output\input.mp4
3 video is [1]
Actual frame 82
File Created : ./Output\input.mp4
4 video is [1]
Actual frame 82
File Created : ./Output\input.mp4
5 video is [1]


## Deployable Code

In [3]:
### TO  COPY:

import pandas as pd
import numpy as np
import cv2 
import mediapipe as mp
import time
import datetime
import matplotlib.pyplot as plt
import skvideo.io
import os
import tensorflow as tf

#Loading model (path will change)
model=tf.keras.models.load_model(r"C:\Users\soory\Documents\GitHub\Project_Moorthi\checkpoint")

mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

def pose_estimation(image,results):
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        return image
    


    
# os.makedirs("Output")
    
start=time.time()
i=0
while(i<5):
    cap = cv2.VideoCapture(0)
    width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out= cv2.VideoWriter('input.mp4', cv2.VideoWriter_fourcc(*'DIVX'),10, (width,height))
    start=time.time()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
           break 
        out.write(frame)
        if time.time() -start >3:
            break     

    cap.release()        
    out.release()      
    cv2.destroyAllWindows()

    outpath=video_array_maker("input.mp4")

    model_input=skvideo.io.vread(outpath)
    prediction=model.predict(np.expand_dims(model_input,axis=0))
    arg_pred=np.argmax(prediction,axis=1)
    i+=1
    print("{} video is {}".format(i,arg_pred))
    start=time.time()